In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

interval_start = -0.4
interval_end = 1
base_domain = np.arange(interval_start, interval_end, 0.0014)

exact_solution = np.float64(0.54105887116969952971)


def function_given(x):
    n = 10
    m = 15
    return m * x * np.e ** (-n) - m * np.e ** (-n * x) + 1 / m


def first_derivative(x):
    n = 10
    m = 15
    return m * n * np.e ** (-n * x) + m * np.e ** (-n)

**Równomierna dystrybucja**

In [ ]:
def even_domain(x_min, x_max, number_of_points):
    return np.linspace(x_min, x_max, number_of_points, dtype=np.double)

**Wizualizacja**

In [ ]:
def visualise(domain=None, nodes_x=None, nodes_y=None, f_y=None, Pn_x=None, title=None, x_size=8, y_size=6,
              plot_title=None,
              filename=None):
    '''
    :param domain: array of float64, of x coordinates of points to visualise
    :param nodes_x: array of x coordinates of nodes
    :param nodes_y: array of f(x) for each x in nodes_x
    :param f_y: array of f(x) for each x in domain
    :param Pn_x: array of Pn(x) for each x in domain
    :param title: a string
    :param x_size: size of a figure in x axis
    :param y_size: size of a figure in y axis
    :param plot_title: title of a plot
    :param filename: name of a file to save a figure
    :return: None
    '''
    plt.figure(figsize=(x_size, y_size))
    if nodes_x is not None and nodes_y is not None:
        plt.scatter(nodes_x, nodes_y, label="Węzły", color="green")
    if f_y is not None:
        plt.plot(domain, f_y, label="Zadana funckcja", color="blue")

    if title is not None:
        plt.plot(domain, Pn_x, label=title, color="red")
    if plot_title is not None:
        plt.title(plot_title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    if filename is not None:
        plt.savefig(f"plots/{filename}.png", dpi=300, format='png')
    plt.show()

In [ ]:
def visualize_functions(given_domain, given_function_values, labels=[], filename=None):
    """
    Plots each function in the given domain on a single plot with distinct colors for each function.

    Parameters:
    - domain (array-like): The domain values.
    - given_function_values (array-like): A 2D array containing the function values for each domain point.
                                   Each row corresponds to a function, and each column corresponds to a domain point.

    Returns:
    - None (displays the plot)
    """
    colors = plt.cm.tab10(np.linspace(0, 1, len(given_function_values)))
    for i, values in enumerate(given_function_values):
        if i < len(labels):
            plt.plot(given_domain, values, label=labels[i], color=colors[i])
        else:
            plt.plot(given_domain, values, color=colors[i])

    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    if filename is not None:
        plt.savefig(f"plots/{filename}.png", dpi=300, format='png')
    plt.show()


**Implementacja funkcji do laboratorium**

In [ ]:
function_given_values = []
for random_name in range(len(base_domain)):
    function_given_values.append(function_given(base_domain[random_name]))

print(len(base_domain))
print(len(function_given_values))


def get_function_values(X):
    result = np.zeros(len(X), dtype=np.double)
    for i in range(len(X)):
        result[i] = function_given(X[i])
    return result


def diff_stop_cond(x1, x0, e):
    return abs(x1 - x0) < e


def value_stop_cond(x, _x, e):
    return abs(function_given(x)) < e

**Metoda Newtona**

In [ ]:
def tangent(x, x0):
    return first_derivative(x0) * (x - x0) + function_given(x0)


def newton_method(x_start, ro, stop_crt, show_steps=False):
    if np.abs(function_given(x_start)) < ro:
        print("early stop for:\nx_start = ", x_start, "\nfunction_given(x_start) = ", function_given(x_start),
              "\nro = ", ro, "\n")
        return 0, x_start
    points = []
    for iter_counter in range(int(10e3)):
        x_next = x_start - function_given(x_start) / first_derivative(x_start)
        if stop_crt(x_next, x_start, ro):
            return iter_counter + 1, x_next
        x_start = x_next
        points.append(x_start)
        if x_start < interval_start or x_start > interval_end:
            print("x_start out of interval:", x_start, "iter_counter:", iter_counter)
            return "poza", x_start
    return "liczba iteracji przekroczona", x_start

**Metoda stycznych**

In [41]:
def secant_method(x0, x1, ro, stop_crt):
    def get_next_sqrt_approx():
        nonlocal x0, x1
        if function_given(x1) - function_given(x0) == 0:
            return False
        return x1 - (x1 - x0) / (function_given(x1) - function_given(x0)) * function_given(x1)

    for iter_counter in range(int(10e3)):
        x0, x1 = x1, get_next_sqrt_approx()
        print("x0", x0, "x1", x1)
        if x1 is False:
            print("dziel przez 0!")
            return "dziel przez 0!", x0
        if x1 < interval_start or x1 > interval_end:
            print("x1 out of interval:", x1, "iter_counter:", iter_counter)
            return "poza", x1
        if stop_crt(x0, x1, ro):
            return iter_counter + 1, x1
    return "liczba iteracji przekroczona", x1


"**Wykonanie ćwiczenia**

In [ ]:
visualise(base_domain, None, None, function_given_values)

In [ ]:
import csv


def save_to_csv(filename, data):
    filename = "errors/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)


ros = np.array([1e-2, 1e-4, 1e-6, 1e-8, 1e-10, 1e-12], dtype=np.float64)

In [ ]:
def perform_newton_experiment(stop_crt):
    global ros
    starting_points = np.arange(-0.4, 1.1, 0.1)
    iteration_results = [["Punkt startowy\Wartość ro", "10E-2", "10E-4", "10E-6", "10E-8", "10E-10", "10E-12"]]
    value_results = [["Punkt startowy\Wartość ro", "10E-2", "10E-4", "10E-6", "10E-8", "10E-10", "10E-12"]]
    for starting_point in starting_points:
        iteration_newline = [starting_point]
        value_newline = [starting_point]
        for ro in ros:
            iteration_number, value_calculated = newton_method(starting_point, ro, stop_crt)
            if not isinstance(iteration_number, int):
                iteration_number = iteration_number
                print("abnormal result for: ", starting_point, ro)
            iteration_newline.append(iteration_number)
            value_newline.append(value_calculated)
        iteration_results.append(iteration_newline)
        value_results.append(value_newline)
    return iteration_results, value_results


iteration_summary, value_summary = perform_newton_experiment(diff_stop_cond)
save_to_csv("newton_diff_iteration", iteration_summary)
save_to_csv("newton_diff_value", value_summary)

iteration_summary, value_summary = perform_newton_experiment(value_stop_cond)
save_to_csv("newton_value_iteration", iteration_summary)
save_to_csv("newton_value_value", value_summary)

In [45]:
iteration_sum = 0

def perform_secant_experiment(stop_crt):
    global ros
    starting_points = np.arange(-0.4, 1.1, 0.1)
    iteration_results = [
        ["x0", "x1", "10E-2", "10E-4", "10E-6", "10E-8", "10E-10", "10E-12"]]
    value_results = [["x0", "x1", "10E-2", "10E-4", "10E-6", "10E-8", "10E-10", "10E-12"]]

    for first_starting_point in [-0.4, 1]:
        for second_starting_point in starting_points:
            if abs(first_starting_point - second_starting_point) < 0.001:
                continue
            iteration_newline = [first_starting_point, second_starting_point]
            value_newline = [first_starting_point, second_starting_point]
            for ro in ros:
                #print("\tro: ", ro)
                iteration_number, value_calculated = secant_method(first_starting_point, second_starting_point, ro,
                                                                   stop_crt)
                if not isinstance(iteration_number, int):
                    iteration_number = iteration_number
                    print("abnormal result for: ", first_starting_point, " ", second_starting_point, ro, "\n")
                else:
                    global iteration_sum
                    iteration_sum += iteration_number
                iteration_newline.append(iteration_number)
                value_newline.append(value_calculated)
            iteration_results.append(iteration_newline)
            value_results.append(value_newline)

    return iteration_results, value_results


'''iteration_summary, value_summary = perform_secant_experiment(diff_stop_cond)
save_to_csv("secant_diff_iteration", iteration_summary)
save_to_csv("secant_diff_value", value_summary)'''
print("------------------------")
iteration_summary, value_summary = perform_secant_experiment(value_stop_cond)
save_to_csv("secant_value_iteration", iteration_summary)
save_to_csv("secant_value_value", value_summary)
print(iteration_sum)

------------------------
x0 -0.30000000000000004 x1 -0.241815175242868
x0 -0.241815175242868 x1 -0.16813128653168952
x0 -0.16813128653168952 x1 -0.10054511900000913
x0 -0.10054511900000913 x1 -0.030674064975649354
x0 -0.030674064975649354 x1 0.03819991601424742
x0 0.03819991601424742 x1 0.10723205651617372
x0 0.10723205651617372 x1 0.17575274474551494
x0 0.17575274474551494 x1 0.24357612301437787
x0 0.24357612301437787 x1 0.30990863222844195
x0 0.30990863222844195 x1 0.37340951537307443
x0 0.37340951537307443 x1 0.4316229813305751
x0 0.4316229813305751 x1 0.4806677020406845
x0 0.4806677020406845 x1 0.5157999939258059
x0 0.5157999939258059 x1 0.534433762216087
x0 0.534433762216087 x1 0.5402661503990419
x0 -0.30000000000000004 x1 -0.241815175242868
x0 -0.241815175242868 x1 -0.16813128653168952
x0 -0.16813128653168952 x1 -0.10054511900000913
x0 -0.10054511900000913 x1 -0.030674064975649354
x0 -0.030674064975649354 x1 0.03819991601424742
x0 0.03819991601424742 x1 0.10723205651617372
x0 0.1